In [1]:
from gensim.models import Word2Vec, KeyedVectors

from entity.Paragraph import Paragraph
from entity.Post import Post
from formula import relevance
from preprocessor.preprocessing import PreprocessPostContent
from utils import idf, search_es, mmr

In [2]:
# load word2vec model
model = KeyedVectors.load_word2vec_format("w2v_model.bin", binary=True)
# load idf_dict
idf_dict = idf.from_csv(file_name="idf.csv")

In [19]:
# give a query string and search from es return top 1000 result
query = "Differences of HashMap HashTable in Java?"
posts = search_es.search(title=query, body=query, size=200)
# create post_obj_list
post_obj_list = []
preprocessor = PreprocessPostContent()
count = 0
for post in posts:
    post_obj_list.append(Post(preprocessor.get_single_para_word_list(post['Title']), post['answers']))

# calculate post score and sort
query_word_list = preprocessor.get_single_para_word_list(query)
for post_obj in post_obj_list:
    post_obj.calculate_score(model, query_word_list, idf_dict)
# select top 5 post create paragraph_obj_list
post_obj_list.sort(reverse=True)
paragraph_obj_list = []
for i in range(5):
    answer_list = post_obj_list[i].answer_list
    for answer in answer_list:
        paragraph_list = preprocessor.getParagraphs(answer['Body'])
        for pos, para in enumerate(paragraph_list, 1):
            paragraph_obj_list.append(
                Paragraph(para, preprocessor.get_single_para_word_list(para), vote_score=answer['Score'],position=pos))
            
# calculate score and sort
relevance_list = []
entropy_list = []
vote_score_list = []
for para_obj in paragraph_obj_list:
    para_obj.cal_relevance(model, query_word_list, idf_dict)
    para_obj.cal_entropy(idf_dict)
    para_obj.cal_semantic_pattern()
    para_obj.cal_format_pattern()
    para_obj.cal_pos_score()
    relevance_list.append(para_obj.relevance_score)
    entropy_list.append(para_obj.infor_entropy)
    vote_score_list.append(para_obj.vote_score)

for para_obj in paragraph_obj_list:
    para_obj.normalized(min(relevance_list), max(relevance_list), min(entropy_list), max(entropy_list),
                        min(vote_score_list), max(vote_score_list))
    para_obj.cal_overall_score()

paragraph_obj_list.sort(reverse=True)
# select top n paragraph and cal mmr
n = 10
paragraph_obj_list = paragraph_obj_list[0:n]
element_obj_list = mmr.cal_mmr(n, paragraph_obj_list, model, idf_dict)
element_obj_list.sort()
# get top k mmr index
k = 5
mmr_index_set = mmr.get_top_k_position(k, element_obj_list)
# loop paragraph_obj_list[mmr_index]
for i in mmr_index_set:
    print(" ".join(paragraph_obj_list[i].word_list))

Hash tables become quite inefficient when there are many collisions While extremely uneven hash distributions are extremely unlikely to arise by chance malicious adversary with knowledge of the hash function may be able to supply information to hash that creates worst-case behavior by causing excessive collisions resulting in very poor performance e.g. denial of service attack 21 In critical applications universal hashing can be used data structure with better worst-case guarantees may be preferable 22
I understand that the number was determined by empirical testing rather than by theoretical analysis thorough theoretical analysis would be difficult However load factor of NUM combined with good hash function is probably sufficient to keep hash chains down to NUM or NUM in the vast majority of cases and that results in fast average lookup times
I think you get mixed up between hash collision and key collision Put it simply hash table consist of collection of linked lists When you add ne